In [1]:
import pandas as pd

In [3]:
df = pd.read_csv(r"C:\Users\Piyush\OneDrive\Desktop\PYTHON\Machine Learning\movie_data_sentiment.csv")

In [4]:
df.head(10)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0
5,Leave it to Braik to put on a good show. Final...,1
6,Nathan Detroit (Frank Sinatra) is the manager ...,1
7,"To understand ""Crash Course"" in the right cont...",1
8,I've been impressed with Chavez's stance again...,1
9,This movie is directed by Renny Harlin the fin...,1


In [5]:
df.loc[0]['review']

'In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich famil

In [6]:
## Take Example for understanding NLTK

In [7]:
docs = 'The sun is shining, The weather is sweet and one and one is two'

In [8]:
import numpy as np

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
count = CountVectorizer()

In [14]:
docs = np.array([docs])

In [15]:
bag = count.fit_transform(docs)

In [16]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [17]:
print(bag.toarray())

[[2 3 2 1 1 1 2 1 1]]


# Term Frequency - Inverse Term Frequency

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer

In [20]:
tfidf = TfidfTransformer(use_idf=True,
                        norm='l2',
                        smooth_idf=True)

In [21]:
print(tfidf.fit_transform(bag).toarray())

[[0.39223227 0.58834841 0.39223227 0.19611614 0.19611614 0.19611614
  0.39223227 0.19611614 0.19611614]]


# Data Preparation

In [29]:
import re
import string

def preprocess(text):
    text = re.sub('<[^>]*>',"",text)
    text = [c for c in text if c not in string.punctuation]
    text = "".join(text)
    return text

In [30]:
df.loc[0]['review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [31]:
preprocess(df.loc[0,'review'][-50:])

'is sevenTitle Brazil Not Available'

In [32]:
## Applying to our data 
df['review'] = df['review'].apply(preprocess)

# Tokenization of documents

In [33]:
from nltk.stem.porter import PorterStemmer

In [34]:
porter = PorterStemmer()

In [35]:
## This will split the words

def tokenizer(text):
    return text.split()

In [36]:
## This will stem the words (running, runs, run, ran = run)

def tokenizer_porter(text):
    return [porter.stem(text) for text in text.split()]

In [37]:
tokenizer_porter('runners like running and so they runs fast')

['runner', 'like', 'run', 'and', 'so', 'they', 'run', 'fast']

In [39]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Piyush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [42]:
from nltk.corpus import stopwords

In [43]:
stop = stopwords.words('english')

In [44]:
[w for w in tokenizer_porter('runners like running and so they runs fast') if w not in stop]

['runner', 'like', 'run', 'run', 'fast']

In [45]:
## Example Over

# Transform text data into Tfidf

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [53]:
tfidf = TfidfVectorizer(tokenizer=tokenizer_porter,
                       use_idf=True,
                       norm='l2',
                       smooth_idf=True)

In [54]:
y = df.sentiment.values

x = tfidf.fit_transform(df.review)

In [55]:
## Train Test Split

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.5)

In [60]:
import pickle

In [61]:
from sklearn.linear_model import LogisticRegressionCV

In [65]:
clf = LogisticRegressionCV(cv=5,
                          scoring='accuracy',
                           random_state=0,
                           n_jobs=-1,
                          verbose=3,
                          max_iter=300)

In [66]:
clf.fit(xtrain,ytrain)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  2.4min remaining:  3.6min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.5min finished


LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=300, multi_class='auto', n_jobs=-1, penalty='l2',
                     random_state=0, refit=True, scoring='accuracy',
                     solver='lbfgs', tol=0.0001, verbose=3)

In [67]:
clf.score(xtest,ytest)

0.89068

In [68]:
#Saving the model

saved_model = open('sentiment_model.sav','wb')

In [69]:
pickle.dump(clf,saved_model)

In [70]:
saved_model.close()

In [72]:
#Opening the model

filename = 'sentiment_model.sav'
saved_clf = pickle.load(open(filename,'rb'))

In [73]:
pred = saved_clf.predict(xtest)